# Projection Comparisons
*Arthur Ryman, lasted updated 2025-09-03*

## Introduction

The goal of this notebook is to analyze the current semantics of some classes that govern the mapping from model
space to scene space. 
My goal is to answer the following questions.

### Q1: Should Projection take a camera location as an init parameter?
The init parameters for Projection class are called scene_x, scene_y, and camera_z,
which makes them look dissimilar, 
but are they in fact components of a camera position vector in model space?

### Q2: Does PuzzleCube3D need the init parameter cube_centre?
The init parameters for PuzzleCube3D includes cube_centre. 
Does it really give additional expressibility or can its effect
be achieved by chaning the projection?

### Q3: Does Puzzle3D need the init parameter cube_one_centre?
Similarly, the init parameters for Puzzle3D include projection and cube_one_centre,
but can cube_one_centre be eliminated by chosing a different projection.

### Manim Voiceover UserWarning

The manim_voiceover module is using a harmless deprecated feature which generates a distracting UserWarning when I import manim.
We can safely ignore it.

In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="manim_voiceover")

# The Projection Class

Source code: 
[projection.py](https://github.com/agryman/instant-insanity/blob/main/src/instant_insanity/core/projection.py)

Here is its init method:

In [2]:
from listings.list_python import lst
from instant_insanity.core.projection import Projection

lst(Projection.__init__)

   1     def __init__(self, scene_x: float = 0.0, scene_y: float = 0.0, camera_z: float = 2.0, scale: float = 1.0) 
   2         self.scene_x = scene_x                                                                                
   3         self.scene_y = scene_y                                                                                
   4         self.camera_z = camera_z                                                                              
   5         self.scale = scale                                                                                    
   6                                                                                                               
   7         scene_origin: Point3D = scene_x * RIGHT + scene_y * UP + camera_z * OUT                               
   8         scene_per_model: float = scale                                                                        
   9         self.conversion = ModelToSceneConversion(scene_origin, scene_per_model)                               
  10                                                                                                               

It is clear from the listing that the code is indeed treating the scene_x, scene_y, and camera_z as the components of a vector.
It remains to confirm that the projection does in fact map this vector to the origin of scene space.

### Using SymPy for Verification

The Projection class, and all other classes I have created for use with Manim, 
use NumPy to represent points in both model space and scene space.
I could therefore write a suite of NumPy test cases to verify that the scene_origin vector in model space 
does indeed get mapped to the origin of scene space.

However, that would only verify the behaviour in a finite number of cases and would not lead to a clearer understanding of the code.
Given that projections are fairly mathematical, it might be feasible to verify the behaviour in all cases by using 
[symbolic exection](https://en.wikipedia.org/wiki/Symbolic_execution) 
of the code.
Therefore, I am going to try using SymPy to verify the code symbolically.

## Symbolic Projection

The Projection class and friends work on NumPy arrays.
I have therefore created parallel versions of the code that work on SymPy objects.

Source code: [symbolic_projection.py](https://github.com/agryman/instant-insanity/blob/main/src/instant_insanity/core/symbolic_projection.py)

For example, here's the init method for the symbolic version of Projection.

In [3]:
import instant_insanity.core.symbolic_projection as sp

lst(sp.Projection.__init__)

   1     def __init__(self,                                                                                        
   2                  scene_x: Scalar = S.Zero,                                                                    
   3                  scene_y: Scalar = S.Zero,                                                                    
   4                  camera_z: Scalar = S.Zero,                                                                   
   5                  scale: Scalar = S.One) -> None:                                                              
   6         self.scene_x = scene_x                                                                                
   7         self.scene_y = scene_y                                                                                
   8         self.camera_z = camera_z                                                                              
   9         self.scale = scale                                                                                    
  10                                                                                                               
  11         scene_origin: Vector = scene_x * UNIT_I + scene_y * UNIT_J + camera_z * UNIT_K                        
  12         scene_per_model: Scalar = scale                                                                       
  13         self.conversion = ModelToSceneConversion(scene_origin, scene_per_model)                               
  14                                                                                                               

Now let's create some SymPy variables to use as init parameters.

In [4]:
from sympy import *
from instant_insanity.core.symbolic_projection import Scalar, Vector

def scalar(name: str) -> Scalar:
    return symbols(name, real=True)

def positive_scalar(name: str) -> Scalar:
    return symbols(name, real=True, positive=True)

def vector(name: str) -> Vector:
    return Matrix(symbols(name + '1:4', real=True))

scene_x = scalar('scene_x')
scene_y = scalar('scene_y')
camera_z = scalar('camera_z')
scale = positive_scalar('scale')

Matrix([scene_x, scene_y, camera_z, scale]).T

Matrix([[scene_x, scene_y, camera_z, scale]])

Projection is an abstract base class so we cannot instantiate it directly.
We need to use one of its concrete subclasses.
Here's the init method for the symbolic version of OrthographicProjection:

In [5]:
lst(sp.OrthographicProjection)

   1 class OrthographicProjection(Projection):                                                                     
   2     """This class models an orthographic projection.                                                          
   3                                                                                                               
   4     Attributes:                                                                                               
   5         u: A unit Vector that specifies the direction of the projection.                                      
   6     """                                                                                                       
   7                                                                                                               
   8     u: Vector                                                                                                 
   9                                                                                                               
  10     def __init__(self, u: Vector, **kwargs) -> None:                                                          
  11         """Initializes an orthographic projection object.                                                     
  12                                                                                                               
  13         Args:                                                                                                 
  14             u: A unit vector that specifies the direction of the projection.                                  
  15                                                                                                               
  16         Raises:                                                                                               
  17             TypeError: if u is not a unit Vector.                                                             
  18             ValueError: if the z-component of u is zero.                                                      
  19         """                                                                                                   
  20         assert isinstance(u, Matrix)                                                                          
  21         assert u.shape == (3, 1)                                                                              
  22                                                                                                               
  23         super().__init__(**kwargs)                                                                            
  24                                                                                                               
  25         norm_u: Scalar = simplify(u.norm())                                                                   
  26         assert norm_u == S.One                                                                                
  27                                                                                                               
  28         u_z: Scalar                                                                                           
  29         _, _, u_z = u                                                                                         
  30         if u_z == S.Zero:                                                                                     
  31             raise ValueError('unit vector z-component is zero')                                               
  32                                                                                                               
  33         self.u = u                                                                                            
  34                                                                                                               
  35     def compute_u(self, model_point: Vector) -> Vec

The model defines a standard orthographic projection.

In [6]:
lst(sp.mk_standard_orthographic_projection)

   1 def mk_standard_orthographic_projection() -> OrthographicProjection:                                          
   2     direction: Vector = Matrix([Rational(3, 2), S.One, Integer(5)])                                           
   3     u: Vector = direction / direction.norm()                                                                  
   4     projection: OrthographicProjection = OrthographicProjection(u,                                            
   5                                                                 scale=Rational(1, 2),                         
   6                                                                 scene_x=Integer(2),                           
   7                                                                 scene_y=Integer(-3),                          
   8                                                                 camera_z=S.One)                               
   9     return projection                                                                                         
  10                                                                                                               

Create an instance of the standard orthographic projection.

In [7]:
sop = sp.mk_standard_orthographic_projection()

sop.u.T

Matrix([[3*sqrt(113)/113, 2*sqrt(113)/113, 10*sqrt(113)/113]])

In [8]:
sop.conversion.scene_origin.T

Matrix([[2, -3, 1]])

Map the scene origin to scene space.

In [9]:
model_camera_origin = sop.project_point(sop.conversion.scene_origin)

model_camera_origin.T

Matrix([[0, 0, 1/2]])

The putative camera origin in scene space does not get mapped to the origin of model space.
I think this is confusing.

Let's look at a totally generic OrthographicProjection and see where it maps the putative camera origin.

An OrthographicProjection takes a unit vector as an init parameter. We need to create a generic unit vector.
The best way to do that is to specify its spherical polar coordinates.

In [10]:
theta = scalar('theta')
phi = scalar('phi')

Matrix([theta, phi]).T

Matrix([[theta, phi]])

In [11]:
u_x = sin(theta) * cos(phi)
u_y = sin(theta) * sin(phi)
u_z = cos(theta)
u = Matrix([u_x, u_y, u_z])

u.T

Matrix([[sin(theta)*cos(phi), sin(phi)*sin(theta), cos(theta)]])

In [12]:
u.norm()

sqrt(sin(phi)**2*sin(theta)**2 + sin(theta)**2*cos(phi)**2 + cos(theta)**2)

In [13]:
simplify(u.norm())

1

In [14]:
trigsimp(u.norm())

1

In [15]:
op = sp.OrthographicProjection(u, scene_x=scene_x, scene_y=scene_y, camera_z=camera_z, scale=scale)

op.u.T

Matrix([[sin(theta)*cos(phi), sin(phi)*sin(theta), cos(theta)]])

In [16]:
op.conversion.scene_origin.T

Matrix([[scene_x, scene_y, camera_z]])

In [17]:
scene_origin = op.conversion.scene_origin

projected_scene_origin = op.project_point(scene_origin)

projected_scene_origin.T

Matrix([[0, 0, camera_z*scale]])

In general, the projection of the scene origin has a nonzero z-component. This seems wrong.
I think it would be more intuitive if the model space scene origin mapped to the origin of scene space.
Furthermore, what we call the scene origing in model space should be called the camera origin as an extension of using camera_z.

I also think that when we create 3D objects, their natural centres should be located at the origin of model space, 
and that further positioning and scaling should be done by the projection by setting the camera origin.

## PerspectiveProjection

Create a generic perspective projection and see where the scene origin gets mapped to.

Here's the initializer:

In [18]:
lst(sp.PerspectiveProjection.__init__)

   1     def __init__(self, viewpoint: Vector, **kwargs) -> None:                                                  
   2         """Initialize the projection.                                                                         
   3                                                                                                               
   4         Args:                                                                                                 
   5             viewpoint: The position of the viewpoint.                                                         
   6                                                                                                               
   7         Raises:                                                                                               
   8             TypeError: if viewpoint is not a Vector.                                                          
   9                                                                                                               
  10         """                                                                                                   
  11         assert isinstance(viewpoint, Matrix)                                                                  
  12         assert viewpoint.shape == (3, 1)                                                                      
  13                                                                                                               
  14         super().__init__(**kwargs)                                                                            
  15         self.viewpoint = viewpoint                                                                            
  16                                                                                                               

In [19]:
viewpoint = vector('v')

viewpoint.T

Matrix([[v1, v2, v3]])

In [20]:
pp = sp.PerspectiveProjection(viewpoint, scene_x=scene_x, scene_y=scene_y, camera_z=camera_z, scale=scale)

pp.viewpoint.T

Matrix([[v1, v2, v3]])

In [21]:
model_scene_origin_pp = pp.project_point(scene_origin)

model_scene_origin_pp.T

Matrix([[0, 0, camera_z*scale]])

In [22]:
pp.conversion.scene_origin.T

Matrix([[scene_x, scene_y, camera_z]])

At least the projections are consistent.

## Action

Look at the code and decide if it make sense to interpret the so-called scene origin as the point in model space that maps to the
origin of scene space. Should this point be called the camera origin?